## Test Case 1: Three Dimensional Deformational Flow

### Import Libraries

In [1]:
import numpy as np
from numpy import linalg as nla
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d.axes3d import Axes3D

# Physical Constants

In [2]:
a = 6.37122*(10**6)  # radius of the Earth (m)
g = 9.80616          # gravitational constant (m s^(-2))
p0 = 100000.0        # reference surface pressure (Pa)
cp = 1004.5          # specific heat capacity of dry air (J kg^(-1) K^(-1))
Rd = 287.0           # gas constant for dry air (J kg^(-1) K^(-1))
kappa = Rd/cp        # ratio of Rd to cp
T0 = 300.0           # isothermal atmosphere temperature (K)
pi = np.pi           # pi
htop = 12000.0

### Simulation Parameterizations

### Test Case Parameterizations

In [3]:
tau = 1036800             # test case duration (s)
b = .2                      # normalized pressure depth of the divergent layer
omega0 = 23000.0 * pi / tau  # Maximum of the vertical pressure velocity in units of Pa s^(-1)

phi_c = 0
lmbda_c1 = 5*pi/6
lmbda_c2 = 7*pi/6
Z_c = 5000.0
R_t = a/2
Z_t = 1000.0

### Coordinate Transformations

The wind fields look nasty in cartesian so I prefer to just calculate them in shperical then go back to cartesian

In [4]:
# convert cartesian to spherical
def c2s(x,y,z):

    ############ calculate r
    r = np.sqrt(x**2 + y**2 + z**2)

    ############ calculate lambda
    lmbda = np.arctan2(y,x)
    
    # map to lambda in [0,2pi)
    lmbda = np.where(y < 0, lmbda + 2*pi, lmbda)

    ############ calculate phi
    phi = np.arctan2(z,np.sqrt(x**2 + y**2))
    
    return r,lmbda,phi



# convert spherical to cartesian
def s2c(r,lmbda,phi):
    
    x = r*np.cos(phi)*np.cos(lmbda)
    y = r*np.cos(phi)*np.sin(lmbda)
    z = r*np.sin(phi)
    
    return [x,y,z]

### Density and Pressure Field

The pressure field for this test case is given by

$$p(r) = p_0 e^{-g\frac{r-a}{R_{d}T_{0}}}$$

which is the hydrostatic pressure of an isothermal atmosphere at $T=T_0$. The perscribed density field is given by

$$\rho(r) = \rho_{0}e^{-(r-a)/H},\quad \text{where } \rho_{0}=\frac{p_{0}}{R_{d}T_{0}} \text{ and } H = \frac{R_{d}T_{0}}{g}$$



In [5]:
# Pressure field
def p(r):
    return p0*np.exp(-g*((r-a)/(Rd*T0)))

# Perscribed density field
def rho(r):
    return (p0/(Rd*T0))*np.exp(-(r-a)*(g/(Rd*T0)))

### Wind Field

Translational latitude defined by

$$\lambda ' = \lambda - 2\pi t/\tau$$

In [6]:
def lmbda_p_t(lmbda,t):
    return lmbda - 2*pi*t/tau

Vertical pressure velocity given by 

$$\omega(\lambda,\phi,p,t) = \omega_0\sin{\lambda'}\cos{\phi}\cos{\frac{2\pi t}{\tau}}s(p)$$

where $s(p)$ is a smooth tapering function taking the vertical velocity to 0 at the top and bottom of the domain and is given by

$$s(p) = 1 + e^{\frac{p_{top}-p_0}{b p_{top}}} - e^{\frac{p-p_0}{b p_{top}}} - e^{\frac{p_{top}-p}{b p_{top}}}$$

where $p_{top} = p(z_{top})$. The vertical velocity then takes the form

$$u_r(\lambda,\phi,p,t) = -\frac{\omega(r,\lambda,\phi,t)}{g\rho}$$

In [7]:
def u_r_t(r,lmbda,phi,t):
    
    p_top = p(a+htop)
    
    s = 1 + np.exp((p_top-p0)/(b*p_top)) + (-np.exp((p(r)-p0)/(b*p_top))) + (-np.exp((p_top-p(r))/(b*p_top)))
    
    omega = omega0*np.sin(lmbda_p_t(lmbda,t))*np.cos(phi)*np.cos(2*pi*t/tau)*s
    
    return - omega/(g*rho(r))

The horizontal zonal $u_\phi$ and meridonal $u_\lambda$ wind velocities are given as the sum of a horizontal deformational component $(u^{(a)}_\phi,u^{(a)}_\lambda)$ and a horizontally divergent component $(u^{(d)}_\phi,u^{(d)}_\lambda)$, which are given by

$$u^{(a)}_\phi(\lambda,\phi,p,t) = \frac{10a}{\tau}\text{sin}^2(\lambda')\sin{(2\phi)}\cos{(\pi t/\tau)}+\frac{2\pi a}{\tau}\cos{(\phi)}$$
$$v^{(a)}_\lambda(\lambda,\phi,p,t) = \frac{10a}{\tau}\sin{(2\lambda')}\cos{(\phi)}\cos{(\pi t/\tau)}$$

$$u^{(d)}_\phi(\lambda,\phi,p,t) = \frac{\omega_0 a}{b p_{top}}\cos{(\lambda')}\text{cos}^2(\phi)\cos(2\pi t/\tau)\big[- e^{\frac{p-p_0}{b p_{top}}} + e^{\frac{p_{top}-p}{b p_{top}}}\big]$$ 
$$u^{(d)}_\lambda(\lambda,\phi,p,t) = 0$$



In [8]:
def u_lmbda_t(r,lmbda,phi,t,c_a,c_d):
    
    p_top = p(a+htop)
    
    # deformational component
    u_a = (10*a/tau)*(np.sin(lmbda_p_t(lmbda,t))**2)*np.sin(2*phi)*np.cos(pi*t/tau)+(2*pi*a/tau)*np.cos(phi)
    
    # divergent component
    u_d = ((omega0*a)/(b*p_top)) * np.cos(lmbda_p_t(lmbda,t)) * (np.cos(phi)**2) * np.cos(2*pi*t/tau) * ((-np.exp((p(r)-p0)/(b*p_top))) + np.exp((p_top-p(r))/(b*p_top)))
    
    return c_a*u_a + c_d*u_d

def u_phi_t(r,lmbda,phi,t):
    
    # deformational component
    v_a = (10*a/tau) * np.sin(2*lmbda_p_t(lmbda,t)) * np.cos(phi)*np.cos(pi*t/tau)
    
    # no divergent component
    
    return v_a


##### Converting to Cartesian Wind Components

We then convert to cartesian by

$$\bar{u} = u_r\hat{r} + u_\lambda\hat{\lambda} + u_\phi\hat{\phi}$$

$$\Rightarrow \bar{u} = u_r(\cos{\phi}\cos{\lambda}\hat{x}+\cos{\phi}\sin{\lambda}\hat{y}+\sin{\phi}\hat{z}) + u_\lambda(\sin{\lambda}\hat{x}+\cos{\lambda}\hat{y}) + u_\phi(-\sin{\phi}\cos{\lambda}\hat{x}-\sin{\phi}\sin{\lambda}\hat{y}+\sin{\phi}\hat{z})$$

$$\Rightarrow \bar{u} = (u_r\cos{\phi}\cos{\lambda}+u_\lambda\sin{\lambda}-u_\phi\sin{\phi}\cos{\lambda})\hat{x} + (u_r\cos{\phi}\sin{\lambda}+u_\lambda\cos{\lambda}-u_\phi\sin{\phi}\sin{\lambda})\hat{y} + (u_r+u_\phi)\sin{\phi}\hat{z}$$

In [9]:
def u_t(r,lmbda,phi,t):
    
    u_r = u_r_t(r,lmbda,phi,t)
    u_lmbda = u_lmbda_t(r,lmbda,phi,t,1,1)
    u_phi = u_phi_t(r,lmbda,phi,t)
    
    u_x = np.cos(phi)*np.cos(lmbda)*u_r + (-np.sin(lmbda)*u_lmbda) + (-np.sin(phi)*np.cos(lmbda)*u_phi)
    u_y = np.cos(phi)*np.sin(lmbda)*u_r + np.cos(lmbda)*u_lmbda + (-np.sin(phi)*np.sin(lmbda)*u_phi)
    u_z = np.sin(phi)*u_r + np.cos(phi)*u_phi
    
    return np.stack((u_x,u_y,u_z))
    

## Tracer Test Cases

In [10]:
def get_q_init(r,lmbda,phi):
    
    r_gc1 = a * np.arccos(np.sin(phi_c)*np.sin(phi) + np.cos(phi_c)*np.cos(phi)*np.cos(lmbda - lmbda_c1))
    r_gc2 = a * np.arccos(np.sin(phi_c)*np.sin(phi) + np.cos(phi_c)*np.cos(phi)*np.cos(lmbda - lmbda_c2))
    
    d_1 = np.minimum((r_gc1/R_t)**2 + (((r - a) - Z_c)/Z_t)**2,1)
    d_2 = np.minimum((r_gc2/R_t)**2 + (((r - a) - Z_c)/Z_t)**2,1)
    
    q_init = 1.0 + (np.cos(pi*d_1) + np.cos(pi*d_2))/2.0
    
    return q_init

In [11]:
# fig = plt.figure(figsize=(20,20))
# ax = fig.add_subplot(2,2,1)
# ax.scatter(rr[1][:,6],rr[2][:,6],c = u_lmbda_t(rr[0],rr[1],rr[2],0,1,0)[:,6])
# ax = fig.add_subplot(2,2,2)
# ax.scatter(rr[1][:,6],rr[2][:,6],c = u_lmbda_t(rr[0],rr[1],rr[2],0,0,1)[:,6])
# ax = fig.add_subplot(2,2,3)
# ax.scatter(rr[1][:,6],rr[2][:,6],c = u_phi_t(rr[0],rr[1],rr[2],0)[:,6])
# ax = fig.add_subplot(2,2,4)
# ax.scatter(rr[1][:,6],rr[2][:,6],c = u_r_t(rr[0],rr[1],rr[2],0)[:,6])

In [12]:
# print(np.max(u_lmbda_t(rr[0],rr[1],rr[2],0,1,0)[:,6]))
# print(np.max(u_lmbda_t(rr[0],rr[1],rr[2],0,0,1)[:,6]))
# print(np.max(u_phi_t(rr[0],rr[1],rr[2],0)[:,6]))
# print(np.max(u_r_t(rr[0],rr[1],rr[2],0)[:,6]))